In [9]:
from binance import Client
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import schedule
import scipy.stats as st
import statistics
import smtplib
import ta
import numpy
import mysql.connector
import pandas_ta as ta
import re
import numpy as np
import datetime

api_key="pTZlrkaCbTEoa17nor24Tw53OWYFlMBQIOmaftSLcjg5lASfhZ0048zcxOZLrBgN"
api_secret="PxcyhPku5uQon1krBa3glze2sF7oZQ3gqb9GRVkh25OFvCtyAkML139l4z9d9BwG"
client=Client(api_key,api_secret)
client.get_account()


{'makerCommission': 10,
 'takerCommission': 10,
 'buyerCommission': 0,
 'sellerCommission': 0,
 'canTrade': True,
 'canWithdraw': True,
 'canDeposit': True,
 'updateTime': 1658290740784,
 'accountType': 'SPOT',
 'balances': [{'asset': 'BTC', 'free': '0.00352160', 'locked': '0.00000000'},
  {'asset': 'LTC', 'free': '0.44100000', 'locked': '0.00000000'},
  {'asset': 'ETH', 'free': '0.02829810', 'locked': '0.00000000'},
  {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNB', 'free': '0.06169163', 'locked': '0.00000000'},
  {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BNT', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'GAS', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'BCC', 'free': '0.00000000', 'locked': '0.00000000'},
  {'asset': 'USDT', 'free': '39.98620489', 'locked

In [10]:
def getdata(symbol, interval, lookback):
    frame=pd.DataFrame(client.get_historical_klines(symbol,interval,lookback))
    frame=frame.iloc[:,:6]
    frame.columns=['Time','Open','High','Low', 'Close','Volume']
    frame=frame.set_index('Time')
    frame.index=pd.to_datetime(frame.index,unit='ms')
    frame = frame.astype(float)
    frame['rsi']=ta.momentum.rsi(frame.Close, window=14)
    frame["EMA_20"] = ta.ema(frame.Close, length=20)
    frame["Close Lag 5"]=frame['Close'].shift(5, axis=0)
    frame["Return 5m"]=(frame['Close']-frame['Close Lag 5'])/frame['Close Lag 5']
    frame['CRYPTO']=symbol
    frame['Date_time'] = frame.index
    frame['Height']=abs(frame['High']-frame['Low'])
    frame['Vol std'] = frame['Volume'].rolling(50).std()
    return frame

In [11]:
#Creating the list of all the cryptos that can be exchange with USDT
exchange_info = client.get_exchange_info()
cryptos=[]
for s in exchange_info['symbols']:
    sy=s['symbol']
    if re.search('..USDT',sy):
        cryptos.append(sy)
        
print(len(cryptos))

406


In [12]:
cry='BTCUSDT'
df_1=getdata(cry,'1m','24 h ago UTC')
df_1['Price_std'] = df_1['Close'].rolling(20).std()
df_1['Price_mean'] = df_1['Close'].rolling(20).mean()
df_1['Boll_up'] = df_1['Price_mean']+2.3*df_1['Price_std']
df_1['Boll_down'] = df_1['Price_mean']-2.3*df_1['Price_std']
df_1['ind'] = range(1, len(df_1) + 1)
df_1['Boll_flag']=''
for i, row in df_1.iterrows():
    if row['ind'] > 100:
            if sum(df_1['Close'][row['ind']-8:row['ind']]>=df_1['Boll_up'][row['ind']-8:row['ind']])<=2 and sum(df_1['Close'][row['ind']-8:row['ind']]<=df_1['Boll_down'][row['ind']-8:row['ind']])<=2:
                df_1.at[i,'Boll_flag']="Yes"
            else:
                df_1.at[i,'Boll_flag']="No"

In [13]:
final_df=pd.DataFrame()
for cry in cryptos:
    try:
        df_1=getdata(cry,'1m','48 h ago UTC')
        df_1['VEMASLOW_30']=ta.ema(df_1.Volume, length=30)
        df_1['VEMAFAST_3']=ta.ema(df_1.Volume, length=3)
        df_1['Vol_Osc_3']=df_1['VEMAFAST_3']-df_1['VEMASLOW_30']
        df_1['Vol_Flag_3_2']=np.where(df_1['Vol_Osc_3']>=2*df_1['Vol std'],"Yes","No")
        macd_df=df_1.ta.macd(close='close', fast=3, slow=26, signal=15, append=True)
        df_1['MACD']=macd_df[macd_df.columns[0]]
        df_1['MACD_H']=macd_df[macd_df.columns[1]]
        df_1['MACD_S']=macd_df[macd_df.columns[2]]
        df_1['ind'] = range(1, len(df_1) + 1)
        df_1['MACD_std'] = df_1['MACD_H'].rolling(50).std()
        df_1['MACD_ratio']=df_1['MACD_H']/df_1['MACD_std']
        #Bollinger bands to confirm break out just began and was not before
        df_1['Price_std'] = df_1['Close'].rolling(20).std()
        df_1['Price_mean'] = df_1['Close'].rolling(20).mean()
        df_1['Boll_up'] = df_1['Price_mean']+2*df_1['Price_std']
        df_1['Boll_down'] = df_1['Price_mean']-2*df_1['Price_std']
        df_1['Boll_flag']=''
        for i, row in df_1.iterrows():
            if row['ind'] > 12:
                    if sum(df_1['Close'][row['ind']-12:row['ind']]>=df_1['Boll_up'][row['ind']-12:row['ind']])<2 and sum(df_1['Close'][row['ind']-12:row['ind']]<=df_1['Boll_down'][row['ind']-12:row['ind']])<=2:
                        df_1.at[i,'Boll_flag']="Yes"
                    else:
                        df_1.at[i,'Boll_flag']="No"

        def boll(df_1):
            if df_1['Close']>=df_1['Boll_up']:
                return "Yes"
            else:
                return "No"
        df_1['Boll']=df_1.apply(boll,axis=1)

        def final_flag(df_1):
                if df_1['Vol_Flag_3_2']=="Yes" and df_1['MACD_H']>3*df_1['MACD_std'] and df_1['MACD']>0 and df_1['Boll']=="Yes" and df_1['Boll_flag']=="Yes" :
                    return "Yes"
                else:
                    return "No"

        #Conditionals
        df_1['filt']=''
        df_1['8_min_ret']=np.nan
        df_1['1_min_ret']=np.nan
        df_1['filt_nextcandle']=''
        for i, row in df_1.iterrows():
            if row['ind'] > 8 and row['ind']+2<len(df_1):
                df_1.at[i,'8_min_ret']=float(abs((df_1['Close'][row['ind']-3]-df_1['Close'][row['ind']-8])/df_1['Close'][row['ind']-8]))
                df_1.at[i,'1_min_ret']=float(abs((row['Close']-row['Open'])/row['Open']))
                if sum(df_1['Height'][row['ind']-8:row['ind']]==0)<=2 and df_1['1_min_ret'][i]>df_1['8_min_ret'][i] :
                    df_1.at[i,'filt']="Yes"
                else:
                    df_1.at[i,'filt']="No"

                if df_1['Close'][row['ind']+1]>df_1['Close'][row['ind']]:
                    df_1.at[i,'filt_nextcandle']="Yes"
                else:
                    df_1.at[i,'filt_nextcandle']="No"

        df_1['final_flag']=df_1.apply(final_flag,axis=1)
        df_f_1=df_1[df_1['final_flag']=="Yes"]
        final_df=pd.concat([final_df,df_f_1])
        print(datetime.datetime.now())
    except:
        print('Not Pssible' +cry)
        

final_df.to_excel('C:/Users/57314/OneDrive - Universidad de los Andes/Binance Automation/fin.xlsx')

2022-07-20 11:58:10.355753
2022-07-20 11:58:21.074416
2022-07-20 11:58:29.671272
Not PssibleBCCUSDT
2022-07-20 11:58:39.850316
2022-07-20 11:58:49.030065
2022-07-20 11:58:56.727196
2022-07-20 11:59:07.862371
2022-07-20 11:59:16.659473
2022-07-20 11:59:25.006731
2022-07-20 11:59:33.511019
2022-07-20 11:59:40.888726
2022-07-20 11:59:49.183872
2022-07-20 11:59:56.432686
2022-07-20 12:00:06.583306
2022-07-20 12:00:15.306799
2022-07-20 12:00:23.935174
Not PssibleVENUSDT
2022-07-20 12:00:32.997707
2022-07-20 12:00:41.431068
Not PssiblePAXUSDT
Not PssibleBCHABCUSDT
Not PssibleBCHSVUSDT
2022-07-20 12:00:50.955369
2022-07-20 12:00:58.001613
2022-07-20 12:01:08.417362
Not PssibleBTTUSDT
Not PssibleUSDSUSDT
2022-07-20 12:01:19.374852
2022-07-20 12:01:27.191751
2022-07-20 12:01:34.801252
2022-07-20 12:01:41.614327
2022-07-20 12:01:49.307988
2022-07-20 12:01:59.023907
2022-07-20 12:02:09.627918
2022-07-20 12:02:17.448371
2022-07-20 12:02:25.792631
2022-07-20 12:02:34.277354
2022-07-20 12:02:42.4857

2022-07-20 12:36:08.986882
2022-07-20 12:36:17.872507
2022-07-20 12:36:26.423303
2022-07-20 12:36:34.371581
2022-07-20 12:36:42.785581
2022-07-20 12:36:50.476827
2022-07-20 12:37:01.380692
2022-07-20 12:37:10.325759
2022-07-20 12:37:18.954218
2022-07-20 12:37:28.079359
2022-07-20 12:37:35.264627
2022-07-20 12:37:44.428408
2022-07-20 12:37:52.849224
2022-07-20 12:38:01.765690
2022-07-20 12:38:09.875648
2022-07-20 12:38:19.655486
2022-07-20 12:38:27.488472
2022-07-20 12:38:35.317163
2022-07-20 12:38:42.503332
2022-07-20 12:38:52.649033
2022-07-20 12:39:01.434350
2022-07-20 12:39:10.495649
2022-07-20 12:39:18.337430
2022-07-20 12:39:27.034542
2022-07-20 12:39:34.217936
2022-07-20 12:39:43.326532
2022-07-20 12:39:52.223329
2022-07-20 12:40:01.571492
2022-07-20 12:40:09.262917
2022-07-20 12:40:18.424697
2022-07-20 12:40:25.558776
Not PssibleRGTUSDT
2022-07-20 12:40:33.084517
2022-07-20 12:40:42.568538
2022-07-20 12:40:52.864426
2022-07-20 12:41:02.454001
2022-07-20 12:41:10.883414
2022-07-2

In [14]:
final_df=pd.DataFrame()
for cry in cryptos:
    #try:
    df_1=getdata(cry,'1m','30 h ago UTC')
    df_1['VEMASLOW_30']=ta.ema(df_1.Volume, length=30)
    df_1['VEMAFAST_3']=ta.ema(df_1.Volume, length=3)
    df_1['VEMAFAST_2']=ta.ema(df_1.Volume, length=2)
    df_1['Vol_Osc_3']=df_1['VEMAFAST_3']-df_1['VEMASLOW_30']
    df_1['Vol_Osc_2']=df_1['VEMAFAST_2']-df_1['VEMASLOW_30']
    df_1['Vol_Flag_3_3']=np.where(df_1['Vol_Osc_3']>=3*df_1['Vol std'],"Yes","No")
    df_1['Vol_Flag_3_2']=np.where(df_1['Vol_Osc_3']>=2*df_1['Vol std'],"Yes","No")
    df_1['Vol_Flag_2_3']=np.where(df_1['Vol_Osc_2']>=3*df_1['Vol std'],"Yes","No")
    df_1['Vol_Flag_2_2']=np.where(df_1['Vol_Osc_3']>=2*df_1['Vol std'],"Yes","No")

    df_1=getdata(cry,'1m','40 h ago UTC')
    macd_df=df_1.ta.macd(close='close', fast=3, slow=26, signal=15, append=True)
    df_1['MACD']=macd_df[macd_df.columns[0]]
    df_1['MACD_H']=macd_df[macd_df.columns[1]]
    df_1['MACD_S']=macd_df[macd_df.columns[2]]
    df_1['ind'] = range(1, len(df_1) + 1)
    df_1['MACD_std']=np.nan
    for i, row in df_1.iterrows():
        if row['ind'] < 150:
            df_1.at[i,'MACD_std']=df_1['MACD_H'][:row['ind']].std()
        else:
            df_1.at[i,'MACD_std']=df_1['MACD_H'][row['ind']-150:row['ind']].std()


    df_1['MACD_ratio']=df_1['MACD_H']/df_1['MACD_std']

    def final_flag(df_1):

            if df_1['Vol_Flag_3_3']=="Yes" or df_1['Vol_Flag_3_2']=="Yes" or df_1['Vol_Flag_2_3']=="Yes"  or df_1['Vol_Flag_2_2']=="Yes" :
                return "Yes"
            else:
                return "No"

    df_1['final_flag']=df_1.apply(final_flag,axis=1)

    df_f_1=df_1[df_1['final_flag']=="Yes"]

    df_1_1=df_1[df_1['Return 5m']>=0.01]
    final_df=pd.concat([final_df,df_1_1])

    print(cry)
    #except:
    print('Not Pssible' +cry)
        

final_df.to_excel('C:/Users/57314/OneDrive - Universidad de los Andes/Binance Automation/VolumeKPI.xlsx')

KeyError: 'Vol_Flag_3_3'

In [ ]:
final_df_macd=pd.DataFrame()
for cry in cryptos:
    try:
        df_1=getdata(cry,'1m','40 h ago UTC')
        macd_df=df_1.ta.macd(close='close', fast=3, slow=26, signal=15, append=True)
        df_1['MACD']=macd_df[macd_df.columns[0]]
        df_1['MACD_H']=macd_df[macd_df.columns[1]]
        df_1['MACD_S']=macd_df[macd_df.columns[2]]
        df_1['ind'] = range(1, len(df_1) + 1)
        df_1['MACD_std']=np.nan
        for i, row in df_1.iterrows():
            if row['ind'] < 150:
                df_1.at[i,'MACD_std']=df_1['MACD_H'][:row['ind']].std()
            else:
                df_1.at[i,'MACD_std']=df_1['MACD_H'][row['ind']-150:row['ind']].std()


        df_1['MACD_ratio']=df_1['MACD_H']/df_1['MACD_std']

        def macd_flag(df_1):
            if df_1['MACD_H']>1.5*df_1['MACD_std'] and df_1['MACD']>0:
                return "Yes"
            else:
                return "No"
        df_1['MACD_Flag']=df_1.apply(macd_flag,axis=1)
        df_1_1=df_1.loc[df_1['MACD_Flag']=="Yes"]
        final_df_macd=pd.concat([final_df_macd,df_1_1])
        print(cry)
    except:
        print('Not Pssible' +cry)
        



In [ ]:
final_df_macd.to_excel('C:/Users/57314/OneDrive - Universidad de los Andes/Binance Automation/MACDkpi.xlsx')

In [ ]:
df_1=getdata('BONDUSDT','1m','24 h ago UTC')
macd_df=df_1.ta.macd(close='close', fast=3, slow=26, signal=15, append=True)
df_1['MACD']=macd_df[macd_df.columns[0]]
df_1['MACD_H']=macd_df[macd_df.columns[1]]
df_1['MACD_S']=macd_df[macd_df.columns[2]]
df_1['ind'] = range(1, len(df_1) + 1)
df_1['MACD_std']=np.nan
for i, row in df_1.iterrows():
    if row['ind'] < 150:
        df_1.at[i,'MACD_std']=df_1['MACD_H'][:row['ind']].std()
    else:
        df_1.at[i,'MACD_std']=df_1['MACD_H'][row['ind']-150:row['ind']].std()

        
df_1['MACD_ratio']=df_1['MACD_H']/df_1['MACD_std']

def macd_flag(df_1):
    if df_1['MACD_H']>1.5*df_1['MACD_std'] and df_1['MACD']>0:
        return "Yes"
    else:
        return "No"
df_1['MACD_Flag']=df_1.apply(macd_flag,axis=1)
df_1_1=df_1.loc[df_1['MACD_Flag']=="Yes"]
df_1_1.head(20)

In [ ]:
df_1=getdata('BONDUSDT','1m','24 h ago UTC')

df_1['Mean_40']=np.nan
df_1['std_40']=np.nan
for i, row in df_1.iterrows():
    if row['ind'] < 40:
        df_1.at[i,'std_40']=df_1['MACD_H'][:row['ind']].std()
        df_1.at[i,'Mean_40']=df_1['MACD_H'][:row['ind']].mean()
    else:
        df_1.at[i,'std_40']=df_1['MACD_H'][row['ind']-40:row['ind']].std()
        df_1.at[i,'Mean_40']=df_1['MACD_H'][row['ind']-40:row['ind']].std()

        
df_1['Bollinger']=df_1['Mean_40']+1.5*df_1['std_40']

def bollinger(df_1):
    if df_1['Close']>df_1['Bollinger']
        return "Yes"
    else:
        return "No"
df_1['bollinger_flag']=df_1.apply(bollinger,axis=1)
df_1_1=df_1.loc[df_1['bollinger_flag']=="Yes"]

In [ ]:
df_1

In [ ]:
df_1=getdata('BONDUSDT','1m','24 h ago UTC')
macd_df=df_1.ta.macd(close='close', fast=3, slow=26, signal=15, append=True)
df_1['MACD']=macd_df[macd_df.columns[0]]
df_1['MACD_H']=macd_df[macd_df.columns[1]]
df_1['MACD_S']=macd_df[macd_df.columns[2]]

for i, row in df_1.iterrows():
    print(i)
    
df_1['MACD_std']=df_1['MACD_H'][:240].std()
print(MACD_std)
def macd_flag(df_1):
    if df_1['MACD_H']>3*MACD_std and df_1['MACD']>0:
        return "Yes"
    else:
        return "No"
df_1['MACD_Flag']=df_1.apply(macd_flag,axis=1)
df_1_1=df_1.loc[df_1['MACD_Flag']=="Yes"]
df_1_1.head(20)

In [ ]:
macd_df

# In resume the best model is the one with 2 times the std
df_1['Vol_Flag_2_2']=np.where(df_1['Vol_Osc_3']>=2*df_1['Vol std'],"Yes","No")

In [ ]:
final_df

# Getting the information

In this part we get two data sets, one for the 15m data and the other of 1m data.
For the 15m data we just need the points with the rsi<30

In [ ]:
df_15=getminutedata('BNBUSDT','15m','2000 h ago UTC')
print(len(df_15))
df_15=df_15.loc[df_15['rsi']<30]
print(len(df_15))
df_1=getminutedata('BNBUSDT','1m','2000 h ago UTC')

# Crating the 15 m label

The idea on this code is to relate the 1m data with the 15m data, for doing that we need to create a column in the 1m data frame where we aprox the time to the 15m.

In [ ]:
df_1['dt15m']=df_1.index
df_1['dt1']=df_1.index
from datetime import datetime
df_1['dt15m']=df_1['dt15m'].astype(str)
df_1.info()
def change_dt(x):
    f = "%Y-%m-%d %H:%M:%S"
    dt = datetime.strptime(x, f)
    if dt.minute < 15:
        m = 0
    elif dt.minute >= 15 and dt.minute < 30:
        m = 15
    elif dt.minute >= 30 and dt.minute < 45:
        m = 30
    elif dt.minute >= 45:
        m = 45
    
    return dt.replace(minute=m, second=0).strftime(f)

df_1.loc[:, 'dt15m'] = df_1['dt15m'].apply(change_dt)


# Creating the merge column in 15 m
We need to create the column for the 15m column to relate the 1m, so we convert the index time to a column.

In [ ]:
df_15['dt15']=df_15.index
df_15['dt15m']=df_15.index
df_15['dt15m']=df_15['dt15m'].astype(str)
df_15.head(30)

# Merge of the data frame
Merging the data, here is important that as we only have the >30 rsi data for 15m. there are nulls in the merged data frame.

For that case we use a method for filling the NaNs called ffill so it fills with the previes values

After that we create a column that compares the 2 dates (index of the 1m and of 15m) we dont want data after 30m of the index of the 15m

In [ ]:
df_final=pd.merge(left=df_1, right=df_15, on='dt15m',how='left')
df_final=df_final.fillna(method='ffill')
df_final.to_excel('C:/Users/57314/OneDrive - Universidad de los Andes/Binance Automation/test.xlsx')

In [ ]:
df_final['dt15']=pd.to_datetime(df_final['dt15'])
df_final['difdt']=abs((df_final['dt1']-df_final['dt15']).astype('timedelta64[m]'))
df_final=df_final.loc[df_final['difdt']<20]

# Grouping the data frame for each of the 15 m labels

Here we will review each of the 15 m labels and search for patterns in them

In [ ]:
dfgrouped=df_final.groupby('dt15')

In [ ]:
path= "C:/Users/57314/OneDrive - Universidad de los Andes/Binance Automation"

In [ ]:
figdic={}
for name, group in dfgrouped:
    fig = go.Figure(data=[go.Candlestick(x=group['dt1'],
                    open=group['Open_x'],
                    high=group['High_x'],
                    low=group['Low_x'],
                    close=group['Close_x'])])

    figdic[str(name)]=fig
    


In [ ]:
listDates=df_final['dt15'].drop_duplicates()
print(listDates)

In [ ]:
for key, value in figdic.items() :
    print (key)
    print(type(key))
    


In [ ]:
figdic["2022-05-10 00:15:00"].show()